In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from smallstations import io

In [28]:
io.inventory()

.gitignore
/test/df.csv


In [31]:
stations_url = 'https://fr.wikipedia.org/wiki/Liste_des_stations_du_m%C3%A9tro_de_Paris'
res = requests.get(stations_url)
soup = BeautifulSoup(res.content,'html5lib')
table = soup.find_all('table')[0]

In [114]:
stations = []
for i, row in enumerate(table.findAll('tr')):
    station = {}
    columns = row.findAll('td')
    try:
        station = columns[0].text.strip().replace('\u200d', '')
        lines_infos = columns[1].findAll('a')
        frequentation = int(columns[6].text.strip().replace('\xa0', '').replace('+0', '').replace(',', ''))
    except IndexError:
        continue
        
    for infos in lines_infos:
        line_name = infos.attrs['title'].replace('\xa0', ' ').replace(' du métro de Paris', '')
        line_url = 'https://fr.wikipedia.org' + infos.attrs['href']

        stations.append({
            'station': station,
            'frequentation': frequentation,
            'line': line_name,
            'line_url': line_url,
        })
        
stations = pd.DataFrame.from_records(stations)
stations.head()

,frequentation,line,line_url,station
0,2417881,Ligne 12,https://fr.wikipedia.org/wiki/Ligne_12_du_m%C3...,Abbesses
1,5124204,Ligne 4,https://fr.wikipedia.org/wiki/Ligne_4_du_m%C3%...,Alésia
2,3666961,Ligne 2,https://fr.wikipedia.org/wiki/Ligne_2_du_m%C3%...,Alexandre Dumas
3,4204705,Ligne 9,https://fr.wikipedia.org/wiki/Ligne_9_du_m%C3%...,Alma - Marceau
4,3780611,Ligne 3,https://fr.wikipedia.org/wiki/Ligne_3_du_m%C3%...,Anatole France


In [115]:
lines = stations.drop_duplicates(subset=['line']).copy()
lines['n'] = lines['line'].apply(lambda s: len(s))
lines.sort_values(by=['n', 'line']).reset_index(drop=True).drop('n', axis=1).reset_index()

,index,frequentation,line,line_url,station
0,0,2972019,Ligne 1,https://fr.wikipedia.org/wiki/Ligne_1_du_m%C3%...,Argentine
1,1,3666961,Ligne 2,https://fr.wikipedia.org/wiki/Ligne_2_du_m%C3%...,Alexandre Dumas
2,2,3780611,Ligne 3,https://fr.wikipedia.org/wiki/Ligne_3_du_m%C3%...,Anatole France
3,3,5124204,Ligne 4,https://fr.wikipedia.org/wiki/Ligne_4_du_m%C3%...,Alésia
4,4,13438906,Ligne 5,https://fr.wikipedia.org/wiki/Ligne_5_du_m%C3%...,Bastille
5,5,2176466,Ligne 6,https://fr.wikipedia.org/wiki/Ligne_6_du_m%C3%...,Bel-Air
6,6,7131674,Ligne 7,https://fr.wikipedia.org/wiki/Ligne_7_du_m%C3%...,Aubervilliers - Pantin - Quatre Chemins
7,7,4454198,Ligne 8,https://fr.wikipedia.org/wiki/Ligne_8_du_m%C3%...,Balard
8,8,4204705,Ligne 9,https://fr.wikipedia.org/wiki/Ligne_9_du_m%C3%...,Alma - Marceau
9,9,1470809,Ligne 10,https://fr.wikipedia.org/wiki/Ligne_10_du_m%C3...,Avenue Émile-Zola


In [60]:
raw = stations[1]['lignes'][0]['row']
raw

<a href="/wiki/Ligne_12_du_m%C3%A9tro_de_Paris" title="Ligne 12 du métro de Paris"><img alt="(12)" data-file-height="92" data-file-width="92" height="16" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Paris_m_12_jms.svg/16px-Paris_m_12_jms.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Paris_m_12_jms.svg/24px-Paris_m_12_jms.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Paris_m_12_jms.svg/32px-Paris_m_12_jms.svg.png 2x" width="16"/></a>

In [63]:
raw.attrs['href']

'/wiki/Ligne_12_du_m%C3%A9tro_de_Paris'

In [41]:
print(str(table)[:2000])

<table class="wikitable sortable center">
<tbody><tr>
<th scope="col">Station</th>
<th scope="col">Ligne(s)</th>
<th scope="col">Rang alpha-<br/>
bétique</th>
<th scope="col">Date d'ouverture</th>
<th scope="col">Situation</th>
<th scope="col">Commune</th>
<th scope="col">Fréquentation annuelle 2015<sup class="reference" id="cite_ref-2"><a href="#cite_note-2"><span class="cite_crochet">[</span>2<span class="cite_crochet">]</span></a></sup></th>
<th scope="col">Correspondance</th>
<th scope="col">Particularité<br/>
(<a href="/wiki/Liste_des_stations_du_m%C3%A9tro_parisien_ayant_chang%C3%A9_de_nom" title="Liste des stations du métro parisien ayant changé de nom">précédent nom</a>)</th>
<th class="unsortable" scope="col">Image</th>
</tr>
<tr>
<td><a href="/wiki/Abbesses_(m%C3%A9tro_de_Paris)" title="Abbesses (métro de Paris)">Abbesses</a></td>
<td align="center"><span style="white-space:nowrap"><span data-sort-value="12!" style="display:none;">‍</span><a href="/wiki/Ligne_12_du_m%C3%A9tro

In [38]:
df = pd.read_html(str(table), flavor='html5lib')[0]
def clean_str(s):
    return s.lower().replace(' ', '')

df.columns = df.iloc[0].apply(lambda s: clean_str(s))
df.drop(0, inplace=True)
df.head()

TypeError: 'NoneType' object is not callable